TODO:
1) EDA
2) Run some training script (ready opensource solution)
3) Analyse results (confusion matrix, some exact cases)

'O': 1146068', geo-nam': 58388, 'org-nam': 48034, 'per-nam': 23790, 'gpe-nam': 20680, 'tim-dat': 12786, 'tim-dow': 11404, 'per-tit': 9800, 'per-fam': 8152, 'tim-yoc': 5290, 'tim-moy': 4262, 'per-giv': 2413, 'tim-clo': 891, 'art-nam': 866, 'eve-nam': 602, 'nat-nam': 300, 'tim-nam': 146, 'eve-ord': 107, 'per-ini': 60, 'org-leg': 60, 'per-ord': 38, 'tim-dom': 10, 'per-mid': 1, 'art-add': 1

In [6]:
from datasets import load_dataset
import numpy as np

In [40]:
ds = load_dataset("ramybaly/conll2012")

conll2012.py: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


0000.parquet:   0%|          | 0.00/9.70M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


0000.parquet:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


0000.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/115812 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/15680 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12217 [00:00<?, ? examples/s]

In [59]:
names=[
                                'O',
                                'B-CARDINAL',
                                'B-DATE',
                                'B-EVENT',
                                'B-FAC',
                                'B-GPE',
                                'B-LANGUAGE',
                                'B-LAW',
                                'B-LOC',
                                'B-MONEY',
                                'B-NORP',
                                'B-ORDINAL',
                                'B-ORG',
                                'B-PERCENT',
                                'B-PERSON',
                                'B-PRODUCT',
                                'B-QUANTITY',
                                'B-TIME',
                                'B-WORK_OF_ART',
                                'I-CARDINAL',
                                'I-DATE',
                                'I-EVENT',
                                'I-FAC',
                                'I-GPE',
                                'I-LANGUAGE',
                                'I-LAW',
                                'I-LOC',
                                'I-MONEY',
                                'I-NORP',
                                'I-ORDINAL',
                                'I-ORG',
                                'I-PERCENT',
                                'I-PERSON',
                                'I-PRODUCT',
                                'I-QUANTITY',
                                'I-TIME',
                                'I-WORK_OF_ART'
                            ]

CLS_MAPPING = {i : ent for i, ent in enumerate(names)}

In [ ]:
# CLS_MAPPING = {
#     0: 'O', #
#     1: 'B-ORG', 2: 'I-ORG',
#     3: 'B-PER', 4: 'I-PER', #
#     5: 'B-GEO', 6: 'I-GEO',
#     7: 'B-GPE', 8: 'I-GPE',
#     9: 'B-TIM', 10: 'I-TIM', #
#     11: 'B-ART', 12: 'I-ART',
#     13: 'B-EVE', 14: 'I-EVE',
#     15: 'B-NAT', 16: 'I-NAT' #
# }

In [43]:
df = ds['train'].to_pandas()

In [44]:
df

,id,tokens,pos_tags,tags
0,0,"[Big, Managers, on, Campus]","[17, 26, 16, 23]","[0, 0, 0, 0]"
1,1,"[In, recent, years, ,, advanced, education, fo...","[16, 17, 26, 3, 17, 23, 16, 26, 43, 41, 12, 17...","[0, 2, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,2,"[With, this, trend, ,, suddenly, the, mature, ...","[16, 12, 23, 3, 31, 12, 17, 26, 16, 26, 40, 12...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,"[In, order, to, attract, this, group, of, seas...","[16, 23, 36, 38, 12, 23, 16, 41, 26, 40, 31, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 27, 27..."
4,4,"[In, response, ,, each, year, over, 1000, matu...","[16, 23, 3, 12, 23, 16, 11, 17, 26, 40, 36, 38...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
115807,115807,"[In, fact, ,, Iraq, has, almost, sunk, into, a...","[16, 23, 3, 24, 43, 31, 41, 16, 12, 23, 16, 17...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
115808,115808,"[The, same, day, ,, the, US, National, Broadca...","[12, 17, 23, 3, 12, 24, 24, 24, 24, 4, 24, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
115809,115809,"[Analysts, here, have, pointed, out, that, Ann...","[26, 31, 42, 41, 34, 16, 24, 28, 23, 10, 12, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
115810,115810,"[Calls, from, within, the, US, for, the, withd...","[26, 16, 16, 12, 24, 16, 12, 23, 16, 26, 16, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [58]:
idx = 6
dict(zip(df.iloc[idx]['tokens'], [CLS_MAPPING[cls_] for cls_ in df.iloc[idx]['tags']]))

{'What': 'O',
 'knowledge': 'O',
 'do': 'O',
 'they': 'O',
 'seek': 'O',
 'that': 'O',
 'can': 'O',
 'not': 'O',
 'be': 'O',
 'found': 'O',
 'in': 'O',
 'society': 'O',
 'at': 'O',
 'large': 'O',
 '?': 'O'}